# Setup and load data

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import re

In [ ]:
# upload files
'''
upload the following:

Comp_data.xlsx - Month 1.csv
Comp_data.xlsx - Month 2.csv
Food Delivery.xlsx - Master Report Monthly (1).csv

'''

from google.colab import files

uploads = files.upload()

Saving Comp_data.xlsx - Month 1.csv to Comp_data.xlsx - Month 1.csv
Saving Comp_data.xlsx - Month 2.csv to Comp_data.xlsx - Month 2.csv
Saving Food Delivery.xlsx - Master Report Monthly (1).csv to Food Delivery.xlsx - Master Report Monthly (1).csv


In [ ]:
comp1 = pd.read_csv('Comp_data.xlsx - Month 1.csv') # comp1 contains the competitions data for month 1
comp2 = pd.read_csv('Comp_data.xlsx - Month 2.csv') # comp2 contains the competitions data for month 2 - month 2 is the latest month
us = pd.read_csv('Food Delivery.xlsx - Master Report Monthly (1).csv') # the table with Mjams data is called 'us' from here on

# Re-Format 'us' table

In order to re-format the table the restaurants and the according dimensions have to be present 1 time each. Than, the features of the 'Unnamed: 4' Column together with the actual values within the 'Month 1' and 'Month 2' Column need to become row based as well and merged into the new table. The idea is to create seperate tables for Month 1 and Month 2 which than can be unioned or merged for analysis as preferred.

## Filtering the table for only relevant ZIP Codes

The stake holders are only interested in a certain area represented by a list of specific ZIP codes.

In [ ]:
us.head() # has one column with multiple properties...

,restaurant ID,name,street,restaurant ZIP,Unnamed: 4,Month 1,Month 2
0,6,Pizzeria Ristorante Ciao,"Barmherzigengasse, 9",1030.0,# Commissionable Orders,225,282
1,6,Pizzeria Ristorante Ciao,"Barmherzigengasse, 9",1030.0,Avg Basket Size €,19.79,21.88
2,6,Pizzeria Ristorante Ciao,"Barmherzigengasse, 9",1030.0,# Negative Comments,0,1
3,6,Pizzeria Ristorante Ciao,"Barmherzigengasse, 9",1030.0,# Positive Comments,1,1
4,6,Pizzeria Ristorante Ciao,"Barmherzigengasse, 9",1030.0,% Kill Rate,4.00%,4.80%


In [ ]:
# change 'restaurant ZIP' data type to integer

us['restaurant ZIP'] = us['restaurant ZIP'].fillna(0).astype('int32') #need to fill na before changing data types

us['restaurant ZIP'].dtypes

dtype('int32')

In [ ]:
# filter for relevant zip codes

relevant_zip_codes = [8010, 8020, 8036, 8041, 8042, 8043, 8045, 8051, 8052, 8053, 8054, 8055, 8063, 8077] #list will contain int32 by default

us_filtered = us[us['restaurant ZIP'].isin(relevant_zip_codes)]

In [ ]:
us_filtered.head()

,restaurant ID,name,street,restaurant ZIP,Unnamed: 4,Month 1,Month 2
510,1505,Mampf,"Herrgottwiesgasse, 51",8020,# Commissionable Orders,699,822
511,1505,Mampf,"Herrgottwiesgasse, 51",8020,Avg Basket Size €,22.5,23.33
512,1505,Mampf,"Herrgottwiesgasse, 51",8020,# Negative Comments,3,4
513,1505,Mampf,"Herrgottwiesgasse, 51",8020,# Positive Comments,1,1
514,1505,Mampf,"Herrgottwiesgasse, 51",8020,% Kill Rate,5.20%,4.60%


## Creating the table for Month 1

In [ ]:
# create a df with only restaurant features and drop duplicates so the granularity is restaurant level

us_formatted_month_1 = us_filtered[['restaurant ID', 'name', 'street', 'restaurant ZIP']].drop_duplicates(keep='first')

In [ ]:
# creating arrays out of every feature that can later be assigned as a new seperate column

us_formatted_month_1['# Commissionable Orders'] = us_filtered[us_filtered['Unnamed: 4'] == '# Commissionable Orders']['Month 1']

avg_basket_size = np.array(us_filtered[us_filtered['Unnamed: 4'] == 'Avg Basket Size €']['Month 1'])

negative_comments = np.array(us_filtered[us_filtered['Unnamed: 4'] == '# Negative Comments']['Month 1'])

positive_comments = np.array(us_filtered[us_filtered['Unnamed: 4'] == '# Positive Comments']['Month 1'])

kill_rate = np.array(us_filtered[us_filtered['Unnamed: 4'] == '% Kill Rate']['Month 1'])

avg_time_accept = np.array(us_filtered[us_filtered['Unnamed: 4'] == 'Avg Time to Accept (s)']['Month 1'])

In [ ]:
# assigning the arrays to the new dataframe

us_formatted_month_1['Avg Basket Size €'] = avg_basket_size
us_formatted_month_1['# Negative Comments'] = negative_comments
us_formatted_month_1['# Positive Comments'] = positive_comments
us_formatted_month_1['% Kill Rate'] = kill_rate
us_formatted_month_1['Avg Time to Accept (s)'] = avg_time_accept

In [ ]:
# creating a seperate 'month' feature

us_formatted_month_1['Month'] = 1

In [ ]:
us_formatted_month_1.head(20) # worked perfectly

,restaurant ID,name,street,restaurant ZIP,# Commissionable Orders,Avg Basket Size €,# Negative Comments,# Positive Comments,% Kill Rate,Avg Time to Accept (s),Month
510,1505,Mampf,"Herrgottwiesgasse, 51",8020,699,22.5,3,1,5.20%,17,1
552,1532,Pizzeria Dolce Vita,"Georgigasse, 26",8020,"1,178",19.12,0,1,6.00%,9,1
618,1560,China Restaurant Mond,"Harrachgasse, 12",8010,230,23.82,1,1,5.40%,19,1
678,1621,Asia Wok Royal,"Eggenberger Allee, 46",8020,692,24.6,3,1,7.70%,27,1
684,1622,Gourmet Express,"Großmarktstraße, 10",8020,365,24.13,2,1,3.90%,20,1
720,1642,Goldene Schwalbe Chinarestaurant,"Lagergasse, 67",8020,106,25.02,1,1,10.00%,34,1
726,1643,Pizza Taxi & Road Burger,"Kärntner Straße, 167",8053,746,26.34,0,1,5.40%,21,1
780,1667,Pizzeria Mama-Rosa,"Wiener Straße, 182 b",8051,336,17.46,9,1,7.50%,26,1
840,1703,China Restaurant Imperium,"Theodor-Körner-Straße, 40",8010,486,24.3,1,1,8.50%,15,1
930,1747,Gambaro Rosso,"St.-Peter-Hauptstraße, 61",8042,248,20.72,1,1,9.30%,18,1


Change '# Commissionable Orders' data type to int

In [ ]:
# remove all non numeric characters

us_formatted_month_1['# Commissionable Orders'] = us_formatted_month_1['# Commissionable Orders'].apply(lambda x: re.sub(r'[^0-9]', '', x))

In [ ]:
# change data type of orders to int

us_formatted_month_1['# Commissionable Orders'] = us_formatted_month_1['# Commissionable Orders'].astype('int')

## Creating a Month 2 table

The exact same steps are repeated for the 'Month 2' cloumn, creating a 2nd df

In [ ]:
us_formatted_month_2 = us_filtered[['restaurant ID', 'name', 'street', 'restaurant ZIP']].drop_duplicates(keep='first')

In [ ]:
us_formatted_month_2['# Commissionable Orders'] = us_filtered[us_filtered['Unnamed: 4'] == '# Commissionable Orders']['Month 2']

avg_basket_size2 = np.array(us_filtered[us_filtered['Unnamed: 4'] == 'Avg Basket Size €']['Month 2'])

negative_comments2 = np.array(us_filtered[us_filtered['Unnamed: 4'] == '# Negative Comments']['Month 2'])

positive_comments2 = np.array(us_filtered[us_filtered['Unnamed: 4'] == '# Positive Comments']['Month 2'])

kill_rate2 = np.array(us_filtered[us_filtered['Unnamed: 4'] == '% Kill Rate']['Month 2'])

avg_time_accept2 = np.array(us_filtered[us_filtered['Unnamed: 4'] == 'Avg Time to Accept (s)']['Month 2'])

In [ ]:
us_formatted_month_2['Avg Basket Size €'] = avg_basket_size2
us_formatted_month_2['# Negative Comments'] = negative_comments2
us_formatted_month_2['# Positive Comments'] = positive_comments2
us_formatted_month_2['% Kill Rate'] = kill_rate2
us_formatted_month_2['Avg Time to Accept (s)'] = avg_time_accept2

In [ ]:
us_formatted_month_2['Month'] = 2

In [ ]:
us_formatted_month_2.head()

,restaurant ID,name,street,restaurant ZIP,# Commissionable Orders,Avg Basket Size €,# Negative Comments,# Positive Comments,% Kill Rate,Avg Time to Accept (s),Month
510,1505,Mampf,"Herrgottwiesgasse, 51",8020,822,23.33,4,1,4.60%,15,2
552,1532,Pizzeria Dolce Vita,"Georgigasse, 26",8020,"1,353",19.95,3,1,5.70%,9,2
618,1560,China Restaurant Mond,"Harrachgasse, 12",8010,375,24.73,0,1,11.20%,20,2
678,1621,Asia Wok Royal,"Eggenberger Allee, 46",8020,734,25.68,8,1,7.30%,28,2
684,1622,Gourmet Express,"Großmarktstraße, 10",8020,410,24.18,0,1,4.40%,18,2


In [ ]:
# remove all non numeric characters

us_formatted_month_2['# Commissionable Orders'] = us_formatted_month_2['# Commissionable Orders'].apply(lambda x: re.sub(r'[^0-9]', '', x))

In [ ]:
# change data type of orders to int

us_formatted_month_2['# Commissionable Orders'] = us_formatted_month_2['# Commissionable Orders'].astype('int')

# Re-Format Comp Tables

## Filtering Comp Tables

Same as for the us table, filtering for relevant ZIP codes will be applied.

**Comp1**

In [ ]:
# subsetting comp1 for relevant zip codes

comp1_filtered = comp1[comp1['zip'].isin(relevant_zip_codes)]

In [ ]:
# checking results

comp1_filtered['zip'].unique()

array([8077, 8063, 8055, 8054, 8053, 8052, 8051, 8045, 8043, 8042, 8041,
       8036, 8020, 8010])

**Comp2**

In [ ]:
# subsetting comp2 for relevant zip codes

comp2_filtered = comp2[comp2['zip'].isin(relevant_zip_codes)]

In [ ]:
# checking results

comp2_filtered['zip'].unique()

array([8010, 8041, 8055, 8020, 8053, 8054, 8052, 8051, 8045, 8042, 8036,
       8043, 8077, 8063])

## Comp1 Table

Since the 'report date' column contains the exact same date for both months it will be dropped. A column with an Integer representing which Month it refers to will be added, similar to the 'us' table.

In [ ]:
comp1.head()

,report_date,lead_id,name,zip,Orders
0,1/6/2021,N111NPQ,Pizza Express Glory Lienz,9900,0
1,1/6/2021,N1NP0RO,Vera Rosa Pizzeria,9813,121
2,1/6/2021,N1OQQN5,Pizzeria Al Pacino,9813,258
3,1/6/2021,O30P3O5N,Ötzi's Fast Food,9800,0
4,1/6/2021,NNRO3RO,Cafe Restaurant Tennishalle,9800,40


In [ ]:
# removing date column

comp1_formatted = comp1_filtered.drop('report_date', axis=1)

In [ ]:
# creating 'month' column

comp1_formatted['month'] = 1

In [ ]:
#checking for duplicates

comp1_formatted.duplicated(subset=['lead_id', 'name']).any()

np.False_

## Comp2 table

Same procedure as for comp1

In [ ]:
# removing date column

comp2_formatted = comp2_filtered.drop('report_date', axis=1)

In [ ]:
# creating 'month' column

comp2_formatted['month'] = 2

In [ ]:
#checking for duplicates

comp2_formatted.duplicated(subset=['lead_id', 'name']).any()

np.False_

In [ ]:
# rename the 'orders' column to fit the comp1 df

comp2_formatted.rename(columns={'orders':'Orders'}, inplace=True, errors='raise')

# Export new data

In [ ]:
# save df's as csv files

us_formatted_month_2.to_csv('us_formatted_month_2.csv', index=False)
us_formatted_month_1.to_csv('us_formatted_month_1.csv', index=False)

comp1_formatted.to_csv('comp_formatted_month_1.csv')
comp2_formatted.to_csv('comp_formatted_month_2.csv')

In [ ]:
# exporting the csv files

files.download('us_formatted_month_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('us_formatted_month_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('comp_formatted_month_1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('comp_formatted_month_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>